# **Cubo mágico**

> Projeto de um cubo mágico utilizando OpenGL e GLFW para a disciplina de Computação Gráfica (SCC0250).

| Estudante 	                | NUSP      |
|-------------------------------|:---------:|
| Gabriel                       |           |
| Italo     	                |      	    |
| João       	                |      	    |
| Lucas     	                |      	    |
| Matheus Henrique de C. Pinto  | 11911104  |

---

# 1. Preparação do ambiente

In [1]:
# Bibliotecas padrão
import glfw
import numpy as np
from OpenGL.GL import *
import OpenGL.GL.shaders

# Custom implementation
from helper import *

# 2. Configuração da janela e shaders

## 2.1 Janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(800, 800, "Cubo", None, None)
glfw.make_context_current(window)

## 2.2 Shaders

In [3]:
# Vertex shader
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [4]:
# Fragment shader
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

# 3. Preparação do programa

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# 3. Preparação do cubo

In [11]:
# Cubos individuais
cubes = []
offset = 0

# Construção do cubo de offsets
for face in range(6):
    face = []
    for row in range(3):
        row = []
        for cube in range(3):
            row.append(offset)
            offset = offset + 24
        face.append(row)
    cubes.append(face)

# Objeto cubo
cube = Cube(cubes)

## 3.1 Organização das faces do cubo

In [12]:
vertices_list = []

corner = get_corner_pieces()
center = get_center_pieces()
edge = get_edge_pieces()

# Face Cima
vertices_list.extend(corner[1])
vertices_list.extend(edge[0][1])
vertices_list.extend(corner[2])

vertices_list.extend(edge[2][1])
vertices_list.extend(center[4])
vertices_list.extend(edge[2][2])

vertices_list.extend(corner[5])
vertices_list.extend(edge[0][2])
vertices_list.extend(corner[6])

# Face Frente
vertices_list.extend(corner[1])
vertices_list.extend(edge[0][1])
vertices_list.extend(corner[2])

vertices_list.extend(edge[1][0])
vertices_list.extend(center[0])
vertices_list.extend(edge[1][3])

vertices_list.extend(corner[0])
vertices_list.extend(edge[0][0])
vertices_list.extend(corner[3])

# Face Direita
vertices_list.extend(corner[2])
vertices_list.extend(edge[2][2])
vertices_list.extend(corner[6])

vertices_list.extend(edge[1][3])
vertices_list.extend(center[3])
vertices_list.extend(edge[1][2])

vertices_list.extend(corner[3])
vertices_list.extend(edge[2][3])
vertices_list.extend(corner[7])

# Face Trás
vertices_list.extend(corner[5])
vertices_list.extend(edge[0][2])
vertices_list.extend(corner[6])

vertices_list.extend(edge[1][1])
vertices_list.extend(center[2])
vertices_list.extend(edge[1][2])

vertices_list.extend(corner[4])
vertices_list.extend(edge[0][3])
vertices_list.extend(corner[7])

# Face Esquerda
vertices_list.extend(corner[1])
vertices_list.extend(edge[2][1])
vertices_list.extend(corner[5])

vertices_list.extend(edge[1][0])
vertices_list.extend(center[1])
vertices_list.extend(edge[1][1])

vertices_list.extend(corner[0])
vertices_list.extend(edge[2][0])
vertices_list.extend(corner[4])

# Face Baixo
vertices_list.extend(corner[0])
vertices_list.extend(edge[0][0])
vertices_list.extend(corner[3])

vertices_list.extend(edge[2][0])
vertices_list.extend(center[5])
vertices_list.extend(edge[2][3])

vertices_list.extend(corner[4])
vertices_list.extend(edge[0][3])
vertices_list.extend(corner[7])

In [13]:
# Construção do array
total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

## 3.2 Transferência dos dados

In [14]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [15]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [16]:
# Bind the position attribute
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

In [17]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [18]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

In [19]:
loc_color = glGetUniformLocation(program, "color")

# 4. Preparação para o loop

In [20]:
# Exibição da janela
glfw.show_window(window)

## 4.1 Utilitários para desenhar as faces pintadas

In [21]:
def draw_cubes(move, before_move=False):
    draw = []

    i = 0
    for color, surface in zip(get_cube_colors(), cube.cubes):
        for rows in surface:
            for cub in rows:
                if ((cub not in move) if before_move else (cub in move)):
                    if (i == 0):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 20, 4)
                        draw.append((cub, 20))

                    elif (i == 1):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub, 4)
                        draw.append((cub, 0))

                    elif (i == 2):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 12, 4)
                        draw.append((cub, 12))

                    elif (i == 3):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 8, 4)
                        draw.append((cub, 8))

                    elif (i == 4):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 4, 4)
                        draw.append((cub, 4))

                    elif (i == 5):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 16, 4)
                        draw.append((cub, 16))
        i = i + 1

    return draw


## 4.2 Utilitário para as faces não pintadas

In [22]:
def draw_blacks(move, draw, before_move=False):
    for surface in cube.cubes:
        for rows in surface:
            for cub in rows:
                for offset in [0, 4, 8, 12, 16, 20]:
                    if ((cub not in move) if before_move else (cub in move)) and (cub, offset) not in draw:
                        glUniform4f(loc_color, 0, 0, 0, 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + offset, 4)


# 5. Programa

## 5.1 Matrizes de transformação

In [23]:
# TODO

def get_static():
    return np.array([1,     0.0,    0,      0.0,
                     0.0,   1.0,    0.0,    0.0,
                     0,     0.0,    1,      0.0,
                     0.0,   0.0,    0.0,    1.0],
                    np.float32)


def get_rotate_z(d):
    return np.array([1.0,   0.0,        0.0,        0.0,
                     0.0,   np.cos(d),  -np.sin(d), 0.0,
                     0.0,   np.sin(d),  np.cos(d),  0.0,
                     0.0,   0.0,        0.0,        1.0],
                    np.float32)


def get_rotate_x(d):
    return np.array([1.0,   0.0,        0.0,        0.0,
                     0.0,   np.cos(d),  -np.sin(d), 0.0,
                     0.0,   np.sin(d),  np.cos(d),  0.0,
                     0.0,   0.0,        0.0,        1.0],
                    np.float32)


def get_rotate_y(d):
    return np.array([np.cos(d),     0.0,    np.sin(d),  0.0,
                     0.0,           1.0,    0.0,        0.0,
                     -np.sin(d),    0.0,    np.cos(d),  0.0,
                     0.0,           0.0,    0.0,        1.0],
                    np.float32)


## 5.2 Loop principal

In [24]:
from numpy import random
import math
d = 0.0
glEnable(GL_DEPTH_TEST)  # importante para 3D

while not glfw.window_should_close(window):

    glfw.poll_events()
    d -= 0.005

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(0.1, 0.1, 0.1, 1.0)

    loc = glGetUniformLocation(program, "mat_transformation")
    
    # Conjunto a ser movido
    move = cube.get_surface_cubes('all')

    draw = []

    # Conjunto antes - afetado apenas por get_static (fora de move)
    glUniformMatrix4fv(loc, 1, GL_TRUE, get_static())
    draw.extend(draw_cubes(move, True))
    draw_blacks(move, draw, True)

    # Conjunto depois - afetado apenas por get_rotate_y (fora de move)
    glUniformMatrix4fv(loc, 1, GL_TRUE, get_rotate_y(d))
    draw.extend(draw_cubes(move, False))
    draw_blacks(move, draw, False)

    glfw.swap_buffers(window)

glfw.terminate()